In [1]:
import argparse
import os
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
model = ""
mfcc = False
silence = False
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# if silence is True:
#     data_dir = os.path.join('.', 'data', 'mini_speech_commands_silence')
# else:
#     zip_path = tf.keras.utils.get_file(
#         origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
#         fname='mini_speech_commands.zip',
#         extract=True,
#         cache_dir='.', cache_subdir='data')
# 
    #  data_dir = os.path('E:\Github\Machine-learning-for-IOT\Lab3\ex2\data\mini_speech_commands')
data_dir = 'E:\Github\Machine-learning-for-IOT\Lab3\ex2\data\mini_speech_commands'

In [4]:
filenames = tf.io.gfile.glob(data_dir + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
if silence is True:
    total = 9000
else:
    total = 8000

In [5]:
num_samples

8000

In [6]:
train_files = filenames[:int(total*0.8)]
val_files = filenames[int(total*0.8): int(total*0.9)]
test_files = filenames[int(total*0.9):]

In [7]:
train_files

<tf.Tensor: shape=(6400,), dtype=string, numpy=
array([b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\up\\b7a6f709_nohash_0.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\go\\e7ea8b76_nohash_7.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\up\\f33660af_nohash_1.wav',
       ...,
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\down\\6cb6eee7_nohash_0.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\left\\68effe85_nohash_0.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\stop\\3a7d8872_nohash_0.wav'],
      dtype=object)>

In [8]:
LABELS = np.array(tf.io.gfile.listdir(data_dir))
LABELS = LABELS[LABELS != 'README.md']

In [9]:
LABELS

array(['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes'],
      dtype='<U9')

In [10]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds


In [11]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [12]:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]

In [13]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

In [14]:
train_files.shape

TensorShape([6400])

In [15]:
# if args.silence is True:
#     units = 9
# else:
#     units = 8

In [17]:
alpha = 0.75

In [18]:
MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(int(alpha*256), activation='relu'),
    keras.layers.Dense(int(alpha*256), activation='relu'),
    keras.layers.Dense(int(alpha*256), activation='relu'),
    keras.layers.Dense(8)
])

In [19]:
#MLPmodel.summary()


In [20]:
CNNmodel = keras.Sequential([
    keras.layers.Conv2D(filters=int(alpha*128), kernel_size=[3,3], strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=int(alpha*128), kernel_size=[3,3], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=int(alpha*128), kernel_size=[3,3], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [21]:
Ds_CNNmodel = keras.Sequential([
    keras.layers.Conv2D(filters=int(alpha*256), kernel_size=[3,3], strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1],use_bias=False),
    keras.layers.Conv2D(filters=int(alpha*256), kernel_size=[1,1], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1],use_bias=False),
    keras.layers.Conv2D(filters=int(alpha*128), kernel_size=[1,1], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [22]:
metrics = [tf.metrics.SparseCategoricalAccuracy()]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
MLPmodel.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

CNNmodel.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

Ds_CNNmodel.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

In [24]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp_MLP/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
MLPmodel.fit(train_ds, epochs=2, callbacks=[cp_callback])

Epoch 1/2
193/200 [===========================>..] - ETA: 0s - loss: 1.6758 - sparse_categorical_accuracy: 0.4109WARNING:tensorflow:From e:\env_ML4IOt\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./callback_test_chkp_MLP\chkp_01\assets
200/200 [==============================] - 14s 68ms/step - loss: 1.6647 - sparse_categorical_accuracy: 0.4150
Epoch 2/2
200/200 [==============================] - 2s 10ms/step - loss: 1.1411 - sparse_categorical_accuracy: 0.6123


In [25]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp_CNN/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
CNNmodel.fit(train_ds, epochs=2, callbacks = cp_callback)

Epoch 1/2
200/200 [==============================] - 15s 77ms/step - loss: 1.5884 - sparse_categorical_accuracy: 0.5330
Epoch 2/2
200/200 [==============================] - 15s 77ms/step - loss: 1.1107 - sparse_categorical_accuracy: 0.6641


In [26]:
train_ds

<ShuffleDataset shapes: ((None, 32, 32, 1), (None,)), types: (tf.float32, tf.int64)>

In [27]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp_Ds_CNN/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
Ds_CNNmodel.fit(train_ds, epochs=2, callbacks = cp_callback)

Epoch 1/2
200/200 [==============================] - 20s 100ms/step - loss: 1.6824 - sparse_categorical_accuracy: 0.5455
Epoch 2/2
200/200 [==============================] - 22s 109ms/step - loss: 1.1750 - sparse_categorical_accuracy: 0.6297


In [28]:
test_acc = MLPmodel.evaluate(test_ds, verbose=0)
print('\n accuracy for MLP :', test_acc)

test_acc = CNNmodel.evaluate(test_ds, verbose=0)
print('\n accuracy for CNN :', test_acc)

test_acc = Ds_CNNmodel.evaluate(test_ds, verbose=0)
print('\n accuracy for Ds_CNN :', test_acc)


 accuracy for MLP : [1.0422528982162476, 0.6512500047683716]

 accuracy for CNN : [0.8661965131759644, 0.7637500166893005]

 accuracy for Ds_CNN : [1.0101279020309448, 0.6537500023841858]


In [29]:
run_model = tf.function(lambda x: MLPmodel(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1,32, 32, 1],tf.float32))
MLPmodel.save('MLP', signatures=concrete_func)

INFO:tensorflow:Assets written to: MLP\assets
